In [2]:
%autosave 60

Autosaving every 60 seconds


In [32]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils,datasets
from tqdm.notebook import tqdm
import torchvision
from torch import nn
import os
import pickle
import sys
from argparse import ArgumentParser

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import tqdm
from torch.nn import functional as fnn
from torch.utils.data import DataLoader
from torchvision import transforms

from utils import NUM_PTS, CROP_SIZE
from utils import ScaleMinSideToSize, CropCenter, TransformByKeys
from utils import ThousandLandmarksDataset
from utils import restore_landmarks_batch, create_submission

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [33]:
PATH='contest01_data'
EXP_NAME='test'

In [34]:


def train(model, loader, loss_fn, optimizer, device):
    model.train()
    train_loss = []
    for batch in tqdm.tqdm(loader, total=len(loader), desc="training..."):
        images = batch["image"].to(device)  # B x 3 x CROP_SIZE x CROP_SIZE
        landmarks = batch["landmarks"]  # B x (2 * NUM_PTS)

        pred_landmarks = model(images).cpu()  # B x (2 * NUM_PTS)
        loss = loss_fn(pred_landmarks, landmarks, reduction="mean")
        train_loss.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return np.mean(train_loss)


def validate(model, loader, loss_fn, device):
    model.eval()
    val_loss = []
    for batch in tqdm.tqdm(loader, total=len(loader), desc="validation..."):
        images = batch["image"].to(device)
        landmarks = batch["landmarks"]

        with torch.no_grad():
            pred_landmarks = model(images).cpu()
        loss = loss_fn(pred_landmarks, landmarks, reduction="mean")
        val_loss.append(loss.item())

    return np.mean(val_loss)


def predict(model, loader, device):
    model.eval()
    predictions = np.zeros((len(loader.dataset), NUM_PTS, 2))
    for i, batch in enumerate(tqdm.tqdm(loader, total=len(loader), desc="test prediction...")):
        images = batch["image"].to(device)

        with torch.no_grad():
            pred_landmarks = model(images).cpu()
        pred_landmarks = pred_landmarks.numpy().reshape((len(pred_landmarks), NUM_PTS, 2))  # B x NUM_PTS x 2

        fs = batch["scale_coef"].numpy()  # B
        margins_x = batch["crop_margin_x"].numpy()  # B
        margins_y = batch["crop_margin_y"].numpy()  # B
        prediction = restore_landmarks_batch(pred_landmarks, fs, margins_x, margins_y)  # B x NUM_PTS x 2
        predictions[i * loader.batch_size: (i + 1) * loader.batch_size] = prediction

    return predictions

In [35]:
train_transforms = transforms.Compose([
    ScaleMinSideToSize((CROP_SIZE, CROP_SIZE)),
    CropCenter(CROP_SIZE),
    TransformByKeys(transforms.ToPILImage(), ("image",)),
    TransformByKeys(transforms.ToTensor(), ("image",)),
    TransformByKeys(transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25]), ("image",)),
])




In [46]:
train_dataset = ThousandLandmarksDataset(os.path.join(PATH, "train"), train_transforms, split="train")
train_dataloader = DataLoader(train_dataset, batch_size=64, num_workers=4, pin_memory=True,
                              shuffle=True, drop_last=True)
val_dataset = ThousandLandmarksDataset(os.path.join(PATH, "train"), train_transforms, split="val")
val_dataloader = DataLoader(val_dataset, batch_size=64, num_workers=4, pin_memory=True,
                            shuffle=False, drop_last=False)

100%|██████████| 64001/64001 [00:04<00:00, 13955.04it/s] 


In [42]:
train_dataset[0]

{'landmarks': tensor([ 74.8592, 118.7112,  73.4729,  ...,  50.3213,  88.2599,  51.7076]),
 'image': tensor([[[-1.9843, -2.0000, -1.3255,  ..., -1.9843, -2.0000, -2.0000],
          [-1.9843, -2.0000, -1.3412,  ..., -2.0000, -2.0000, -2.0000],
          [-1.9843, -2.0000, -1.3255,  ..., -2.0000, -2.0000, -2.0000],
          ...,
          [-2.0000, -1.9843, -1.8275,  ..., -1.9686, -2.0000, -2.0000],
          [-2.0000, -1.9843, -1.8118,  ..., -1.9843, -2.0000, -2.0000],
          [-2.0000, -1.9843, -1.7804,  ..., -1.9686, -2.0000, -2.0000]],
 
         [[-2.0000, -2.0000, -1.4824,  ..., -2.0000, -2.0000, -2.0000],
          [-2.0000, -2.0000, -1.5294,  ..., -2.0000, -2.0000, -2.0000],
          [-2.0000, -2.0000, -1.5608,  ..., -2.0000, -2.0000, -2.0000],
          ...,
          [-1.9843, -2.0000, -1.9529,  ..., -2.0000, -2.0000, -2.0000],
          [-1.9843, -2.0000, -1.9373,  ..., -2.0000, -2.0000, -2.0000],
          [-1.9843, -2.0000, -1.9373,  ..., -1.9843, -2.0000, -2.0000]],
 
 

In [43]:

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [41]:

model = torchvision.models.vgg11(pretrained=True)
model.requires_grad_(False)

model.classifier[6] = nn.Linear(4096,971,bias=True)
model.classifier.requires_grad_(True)

model.to(device)

Downloading: "https://download.pytorch.org/models/vgg11-bbd30ac9.pth" to /home/ilyua/.cache/torch/hub/checkpoints/vgg11-bbd30ac9.pth


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [44]:


optimizer = optim.Adam(model.parameters(), lr=0.001, amsgrad=True)
loss_fn = fnn.mse_loss
EPOCHS=100

In [45]:
os.makedirs("runs", exist_ok=True)
best_val_loss = np.inf
for epoch in range(EPOCHS):
    train_loss = train(model, train_dataloader, loss_fn, optimizer, device=device)
    val_loss = validate(model, val_dataloader, loss_fn, device=device)
    print("Epoch #{:2}:\ttrain loss: {:5.2}\tval loss: {:5.2}".format(epoch, train_loss, val_loss))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        with open(os.path.join("runs", f"{EXP_NAME}_best.pth"), "wb") as fp:
            torch.save(model.state_dict(), fp)

# 3. predict
test_dataset = ThousandLandmarksDataset(os.path.join(PATH, "test"), train_transforms, split="test")
test_dataloader = DataLoader(test_dataset, batch_size=128, num_workers=4, pin_memory=True,
                             shuffle=False, drop_last=False)

with open(os.path.join("runs", f"{EXP_NAME}_best.pth"), "rb") as fp:
    best_state_dict = torch.load(fp, map_location="cpu")
    model.load_state_dict(best_state_dict)

test_predictions = predict(model, test_dataloader, device)
with open(os.path.join("runs", f"{EXP_NAME}_test_predictions.pkl"), "wb") as fp:
    pickle.dump({"image_names": test_dataset.image_names,
                 "landmarks": test_predictions}, fp)

create_submission(args.data, test_predictions, os.path.join("runs", f"{EXP_NAME}_submit.csv"))


training...:   0%|          | 0/399 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 1.95 GiB total capacity; 616.99 MiB already allocated; 374.31 MiB free; 736.00 MiB reserved in total by PyTorch)

In [23]:
model = torchvision.models.vgg11()

In [29]:
model.classifier[6] = nn.Linear(4096,971)

In [30]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

# Submission

In [11]:
test_dataset = datasets.ImageFolder('contest01_data/test/')

In [16]:
points = pd.read_csv('contest01_data/test/test_points.csv',sep='\t')

In [ ]:
DataLoader(dataset, batch_size=1,

In [ ]:
def make_submission(model,test_dataloader,points,submission_name='test'):
    for img_name in tqdm(points['file_name'].values):
        predicted_landmarks = model(torch.Tensor(io.read_image(img_name)))
        ####